## Model Registry

In [ ]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
registry.list_models()

registry.print_model_details("fai-rtdetr-l-obj365")

model_info = registry.get_model_info("fai-rtdetr-l-obj365")

## AutoDatasets

In [ ]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.ports import DatasetLayout, DatasetSplitType, Task

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_datasets = AutoDataset(dataset_name="football.zip", task=task, layout=layout, datasets_root_dir="../data")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_datasets.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_datasets.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

## Auto Model

In [ ]:
from focoos.auto_model import AutoModel

model = AutoModel.from_pretrained("fai-rtdetr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

## Trainer

In [ ]:
from focoos.ports import TrainerArgs
from focoos.trainer.trainer import FocoosTrainer

args = TrainerArgs(
    run_name="pippo",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=8,
    eval_period=50,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
)


trainer = FocoosTrainer(args=args, model=model, data_train=train_dataset, data_val=valid_dataset)

trainer.train()